In [2]:
# Import Required Libraries
import warnings

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

# Set display options for better output
pd.set_option("display.precision", 4)
np.set_printoptions(precision=4, suppress=True)

In [3]:
# Create the Decision Matrix
criteria = [
    "Cost per Unit",
    "Delivery Time",
    "Quality Score",
    "Innovation Index",
    "Sustainability Score",
]
alternatives = ["GTS", "ISC", "RCI", "ETP"]

# Decision matrix with actual values
# Construction cost in Million PHP, others on 1-10 scale
decision_matrix = np.array(
    [
        [4550, 12, 94.2, 6.5, 72],
        [5230, 8, 91.8, 9.2, 68],
        [5870, 15, 98.5, 5.8, 85],
        [6120, 18, 89.3, 8.7, 96],
    ]
)

# Create DataFrame for better visualization
df_decision = pd.DataFrame(
    decision_matrix, index=alternatives, columns=criteria
)

print("Decision Matrix:")
print("=" * 50)
print(df_decision)
print(f"\nMatrix shape: {decision_matrix.shape}")
print(f"Alternatives: {len(alternatives)}")
print(f"Criteria: {len(criteria)}")

Decision Matrix:
     Cost per Unit  Delivery Time  Quality Score  Innovation Index  \
GTS         4550.0           12.0           94.2               6.5   
ISC         5230.0            8.0           91.8               9.2   
RCI         5870.0           15.0           98.5               5.8   
ETP         6120.0           18.0           89.3               8.7   

     Sustainability Score  
GTS                  72.0  
ISC                  68.0  
RCI                  85.0  
ETP                  96.0  

Matrix shape: (4, 5)
Alternatives: 4
Criteria: 5


In [4]:
# Define Criteria Weights and Optimization Direction
weights = np.array([0.25, 0.20, 0.30, 0.15, 0.10])

# Optimization direction: 1 for maximize, -1 for minimize
optimization_direction = np.array(
    [-1, -1, 1, 1, 1]
)

print("Criteria Information:")
print("=" * 50)
for i, (criterion, weight, direction) in enumerate(
    zip(criteria, weights, optimization_direction)
):
    opt_dir = "Minimize" if direction == -1 else "Maximize"
    print(f"{criterion:<25}: Weight = {weight:.2f}, Direction = {opt_dir}")

print(f"\nSum of weights: {weights.sum():.2f}")

# Verify weights sum to 1
assert abs(weights.sum() - 1.0) < 1e-10, "Weights must sum to 1.0"
print("✓ Weights validation passed!")

Criteria Information:
Cost per Unit            : Weight = 0.25, Direction = Minimize
Delivery Time            : Weight = 0.20, Direction = Minimize
Quality Score            : Weight = 0.30, Direction = Maximize
Innovation Index         : Weight = 0.15, Direction = Maximize
Sustainability Score     : Weight = 0.10, Direction = Maximize

Sum of weights: 1.00
✓ Weights validation passed!


In [5]:
# Step 1: Normalize the Decision Matrix
def normalize_matrix(matrix, optimization_direction):
    """
    Normalize the decision matrix using vector normalization.
    For minimization criteria, we use 1/value before normalization.
    """
    normalized = matrix.copy().astype(float)

    for j in range(matrix.shape[1]):
        if optimization_direction[j] == -1:  # Minimize (like price)
            # For minimization: use reciprocal then normalize
            normalized[:, j] = 1 / matrix[:, j]

        # Vector normalization
        column_norm = np.sqrt(np.sum(normalized[:, j] ** 2))
        normalized[:, j] = normalized[:, j] / column_norm

    return normalized


# Apply normalization
normalized_matrix = normalize_matrix(decision_matrix, optimization_direction)

# Create DataFrame for better visualization
df_normalized = pd.DataFrame(
    normalized_matrix, index=alternatives, columns=criteria
)

print("Normalized Decision Matrix:")
df_normalized

Normalized Decision Matrix:


,Cost per Unit,Delivery Time,Quality Score,Innovation Index,Sustainability Score
GTS,0.5862,0.4803,0.5037,0.4229,0.4444
ISC,0.5100,0.7205,0.4908,0.5986,0.4197
RCI,0.4544,0.3843,0.5267,0.3774,0.5246
ETP,0.4358,0.3202,0.4775,0.5661,0.5925


In [6]:
# Step 2: Apply Weights to Create Weighted Normalized Matrix
weighted_matrix = normalized_matrix * weights

# Create DataFrame for weighted matrix
df_weighted = pd.DataFrame(
    weighted_matrix, index=alternatives, columns=criteria
)

print("Weighted Normalized Matrix:")
df_weighted

Weighted Normalized Matrix:


,Cost per Unit,Delivery Time,Quality Score,Innovation Index,Sustainability Score
GTS,0.1465,0.0961,0.1511,0.0634,0.0444
ISC,0.1275,0.1441,0.1473,0.0898,0.0420
RCI,0.1136,0.0769,0.1580,0.0566,0.0525
ETP,0.1089,0.0640,0.1432,0.0849,0.0593


In [7]:
# Step 3: Calculate Concordance Index
def calculate_concordance_matrix(weighted_matrix, weights):
    """
    Calculate concordance matrix for all pairs of alternatives.
    C(i,k) = sum of weights where alternative i >= alternative k
    """
    n_alternatives = weighted_matrix.shape[0]
    concordance_matrix = np.zeros((n_alternatives, n_alternatives))

    for i in range(n_alternatives):
        for k in range(n_alternatives):
            if i != k:
                # Find criteria where alternative i >= alternative k
                concordant_criteria = weighted_matrix[i] >= weighted_matrix[k]
                # Sum weights of concordant criteria
                concordance_matrix[i, k] = np.sum(weights[concordant_criteria])

    return concordance_matrix


# Calculate concordance matrix
concordance_matrix = calculate_concordance_matrix(weighted_matrix, weights)

# Create DataFrame for better visualization
df_concordance = pd.DataFrame(
    concordance_matrix, index=alternatives, columns=alternatives
)

print("Concordance Matrix:")
df_concordance

Concordance Matrix:


,GTS,ISC,RCI,ETP
GTS,0.00,0.65,0.60,0.75
ISC,0.35,0.00,0.60,0.90
RCI,0.40,0.40,0.00,0.75
ETP,0.25,0.10,0.25,0.00


In [8]:
# Step 4: Calculate Discordance Index
def calculate_discordance_matrix(weighted_matrix):
    """
    Calculate discordance matrix for all pairs of alternatives.
    D(i,k) = max difference where i < k / max overall difference
    """
    n_alternatives = weighted_matrix.shape[0]
    discordance_matrix = np.zeros((n_alternatives, n_alternatives))

    # Find maximum difference across all criteria and pairs
    max_diff_overall = 0
    for i in range(n_alternatives):
        for k in range(n_alternatives):
            if i != k:
                max_diff = np.max(
                    np.abs(weighted_matrix[i] - weighted_matrix[k])
                )
                max_diff_overall = max(max_diff_overall, max_diff)

    for i in range(n_alternatives):
        for k in range(n_alternatives):
            if i != k:
                # Find criteria where alternative i < alternative k (discordant)
                discordant_criteria = weighted_matrix[i] < weighted_matrix[k]

                if np.any(discordant_criteria):
                    # Maximum difference among discordant criteria
                    max_diff_discordant = np.max(
                        np.abs(weighted_matrix[i] - weighted_matrix[k])[
                            discordant_criteria
                        ]
                    )
                    discordance_matrix[i, k] = (
                        max_diff_discordant / max_diff_overall
                    )
                else:
                    discordance_matrix[i, k] = 0

    return discordance_matrix


# Calculate discordance matrix
discordance_matrix = calculate_discordance_matrix(weighted_matrix)

# Create DataFrame for better visualization
df_discordance = pd.DataFrame(
    discordance_matrix, index=alternatives, columns=alternatives
)

print("Discordance Matrix:")
df_discordance

Discordance Matrix:


,GTS,ISC,RCI,ETP
GTS,0.0000,0.60,0.1002,0.2682
ISC,0.2380,0.00,0.1343,0.2159
RCI,0.4116,0.84,0.0000,0.3535
ETP,0.4696,1.00,0.1843,0.0000


In [22]:
# Step 5: Determine Outranking Relations
def determine_outranking_relations(
    concordance_matrix,
    discordance_matrix,
    concordance_threshold=0.5,
    discordance_threshold=0.5,
):
    """
    Determine outranking relations based on concordance and discordance thresholds.
    Alternative i outranks alternative k if:
    - C(i,k) >= concordance_threshold AND
    - D(i,k) <= discordance_threshold
    """
    n_alternatives = concordance_matrix.shape[0]
    outranking_matrix = np.zeros((n_alternatives, n_alternatives), dtype=int)

    for i in range(n_alternatives):
        for k in range(n_alternatives):
            if i != k:
                concordance_condition = (
                    concordance_matrix[i, k] >= concordance_threshold
                )
                discordance_condition = (
                    discordance_matrix[i, k] <= discordance_threshold
                )

                if concordance_condition and discordance_condition:
                    outranking_matrix[i, k] = 1

    return outranking_matrix


# Set thresholds
concordance_threshold = 0.5
discordance_threshold = 0.5

print("Thresholds:")
print(f"- Concordance threshold: {concordance_threshold}")
print(f"- Discordance threshold: {discordance_threshold}")
print()

# Calculate outranking relations
outranking_matrix = determine_outranking_relations(
    concordance_matrix,
    discordance_matrix,
    concordance_threshold,
    discordance_threshold,
)

# Create DataFrame for outranking matrix
df_outranking = pd.DataFrame(
    outranking_matrix, index=alternatives, columns=alternatives
)

print("Outranking Matrix (1 = outranks, 0 = does not outrank):")
df_outranking

Thresholds:
- Concordance threshold: 0.5
- Discordance threshold: 0.5

Outranking Matrix (1 = outranks, 0 = does not outrank):


,GTS,ISC,RCI,ETP
GTS,0,0,1,1
ISC,0,0,1,1
RCI,0,0,0,1
ETP,0,0,0,0


In [24]:
# Analyze Results and Find the Kernel (Best Alternatives)
def analyze_outranking_results(outranking_matrix, alternatives):
    """
    Analyze the outranking matrix to find dominated and non-dominated alternatives.
    The kernel consists of alternatives that are not outranked by any other alternative.
    """
    n_alternatives = len(alternatives)

    # Count how many alternatives each one outranks
    outranks_count = np.sum(outranking_matrix, axis=1)

    # Count how many alternatives outrank each one
    outranked_by_count = np.sum(outranking_matrix, axis=0)

    # Find the kernel (non-dominated alternatives)
    kernel = []
    for i in range(n_alternatives):
        if outranked_by_count[i] == 0:  # Not outranked by any alternative
            kernel.append(i)

    return outranks_count, outranked_by_count, kernel


# Analyze results
outranks_count, outranked_by_count, kernel_indices = (
    analyze_outranking_results(outranking_matrix, alternatives)
)

# Create summary DataFrame
summary_data = {
    "Alternative": alternatives,
    "Outranks Count": outranks_count,
    "Outranked By Count": outranked_by_count,
    "In Kernel": [
        "Yes" if i in kernel_indices else "No"
        for i in range(len(alternatives))
    ],
}

df_summary = pd.DataFrame(summary_data)

print("ELECTRE I Results Summary:")
print(df_summary)

print("Kernel (Best Alternatives):")
if kernel_indices:
    for i in kernel_indices:
        print(f"{alternatives[i]}")
else:
    print("No alternatives in the kernel (all are dominated)")

ELECTRE I Results Summary:
  Alternative  Outranks Count  Outranked By Count In Kernel
0         GTS               2                   0       Yes
1         ISC               2                   0       Yes
2         RCI               1                   2        No
3         ETP               0                   3        No
Kernel (Best Alternatives):
GTS
ISC
